In [ ]:
import pandas as pd
import yfinance as yf
import time
import numpy as np
import matplotlib.pyplot as plt


# Define start and end dates
start = pd.to_datetime('2013-01-01')
end = pd.to_datetime('2018-01-01')

# Create an empty dictionary to store dataframes
stock_data = {}

# Define the stock symbols (Yahoo Finance uses different symbols compared to Quandl)
stock_symbols = ['JPM', 'C', 'BAC', 'WFC']

# Fetch data from Yahoo Finance
for stock_symbol in stock_symbols:
    try:
        stock_df = yf.download(stock_symbol, start=start, end=end)

        # Process the stock data
        stock_df['Normed Return'] = stock_df['Adj Close'] / stock_df.iloc[0]['Adj Close']

        # Store the dataframe in the dictionary
        stock_data[stock_symbol] = stock_df

    except Exception as e:
        print(f"Error retrieving data for symbol: {stock_symbol}. Error message: {e}")

# Access and display dataframes (similar to the original loop)
for stock_name, stock_df in stock_data.items():
    print(f"\n{stock_name} DataFrame Head:")
    print(stock_df.head())

allocations = [0.3, 0.2, 0.4, 0.1]
for (stock_name, stock_df), allocation in zip(stock_data.items(), allocations):
    stock_df['Allocation'] = stock_df['Normed Return'] * allocation

# Display the head of the JPM dataframe with allocation
print("\nJPM DataFrame with Allocation Head:")
print(stock_data['JPM'].head())

# Calculate and add position values to each dataframe
for stock_df in stock_data.values():
    stock_df['Position Values'] = stock_df['Allocation'] * 1000000

# Display the head of the JPM dataframe with allocation and position values
print("\nJPM DataFrame with Allocation and Position Values Head:")
print(stock_data['JPM'].head())


In [ ]:
# Extract the position values for each stock
jpm = stock_data['JPM']
citi = stock_data['C']
bofa = stock_data['BAC']
wfc = stock_data['WFC']

# Combine the position values into a single DataFrame and calculate the total position
portfolio_val = pd.concat([jpm['Position Values'], citi['Position Values'], bofa['Position Values'], wfc['Position Values']], axis=1)
portfolio_val.columns = ['JPM Pos', 'CITI Pos', 'BOFA Pos', 'WFC Pos']
portfolio_val['Total Pos'] = portfolio_val.sum(axis=1)

# Display the head of the portfolio valuation DataFrame
print("\nPortfolio Valuation DataFrame Head:")
print(portfolio_val.head())

In [ ]:
# Plot the total portfolio value
portfolio_val['Total Pos'].plot(figsize=(10, 8))
plt.title('Total Portfolio Value')
plt.xlabel('Date')
plt.ylabel('Portfolio Value')
plt.show()

# Plot the individual stock position values
portfolio_val.drop('Total Pos', axis=1).plot(kind='line', figsize=(10, 8))
plt.title('Individual Stock Position Values')
plt.xlabel('Date')
plt.ylabel('Position Value')
plt.show()


In [ ]:
portfolio_val['Daily Return'] = portfolio_val['Total Pos'].pct_change(1)
portfolio_val

In [ ]:
# Calculate the cumulative return
cum_ret = 100 * (portfolio_val['Total Pos'][-1] / portfolio_val['Total Pos'][0] - 1)
print('Our cumulative return is {} percent!'.format(cum_ret))

In [ ]:
portfolio_val['Daily Return'].mean()

In [ ]:
portfolio_val['Daily Return'].std()

In [ ]:
SR = portfolio_val['Daily Return'].mean()/portfolio_val['Daily Return'].std()
SR

In [ ]:
ASR = (252**0.5)*SR
ASR

In [ ]:


# Extract and combine the adjusted close prices into a single DataFrame for daily returns
jpm_close = jpm['Adj Close']
citi_close = citi['Adj Close']
bofa_close = bofa['Adj Close']
wfc_close = wfc['Adj Close']

# Combine daily close prices into a single DataFrame
stocks = pd.concat([jpm_close, citi_close, bofa_close, wfc_close], axis=1)
stocks.columns = ['jpm', 'citi', 'bofa', 'wfc']

# Display the head of the daily returns DataFrame
print("\nDaily Returns DataFrame Head:")
print(stocks.head())

In [ ]:
log_ret = np.log(stocks/stocks.shift(1))
log_ret.head()

In [ ]:
#calculate the log return mean of each stock
log_ret.mean() * 252

In [ ]:
# Compute pairwise covariance of columns
log_ret.cov()*252

In [ ]:
np.random.seed(101)

# Stock Columns
print('Stocks')
print(stocks.columns)
print('\n')

# Create Random Weights
print('Creating Random Weights')
weights = np.array(np.random.random(4))
print(weights)
print('\n')

# Rebalance Weights
print('Rebalance to sum to 1.0')
weights = weights / np.sum(weights)
print(weights)
print('\n')

# Expected Return
print('Expected Portfolio Return')
exp_ret = np.sum(log_ret.mean() * weights) *252
print(exp_ret)
print('\n')

# Expected Variance
print('Expected Volatility')
exp_vol = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov() * 252, weights)))
print(exp_vol)
print('\n')

# Sharpe Ratio
SR = exp_ret/exp_vol
print('Sharpe Ratio')
print(SR)

In [ ]:
num_ports = 20000

all_weights = np.zeros((num_ports,len(stocks.columns)))
ret_arr = np.zeros(num_ports)
vol_arr = np.zeros(num_ports)
sharpe_arr = np.zeros(num_ports)

for ind in range(num_ports):

    # Create Random Weights
    weights = np.array(np.random.random(4))

    # Rebalance Weights
    weights = weights / np.sum(weights)
    
    # Save Weights
    all_weights[ind,:] = weights

    # Expected Return
    ret_arr[ind] = np.sum((log_ret.mean() * weights) *252)

    # Expected Variance
    vol_arr[ind] = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov() * 252, weights)))

    # Sharpe Ratio
    sharpe_arr[ind] = ret_arr[ind]/vol_arr[ind]

In [ ]:
sharpe_arr.max()

In [ ]:
sharpe_arr.argmax()

In [ ]:
all_weights[1248,:]

In [ ]:
max_sr_ret = ret_arr[1248]
max_sr_vol = vol_arr[1248]
plt.figure(figsize=(17,9))
plt.scatter(vol_arr,ret_arr,c=sharpe_arr,cmap='plasma')
plt.colorbar(label='Sharpe Ratio')
plt.xlabel('Volatility')
plt.ylabel('Return')
# Add red dot for max SR
plt.scatter(max_sr_vol,max_sr_ret,c='red',s=50,edgecolors='black')